### Shared Word File Process

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # always must be False in this part
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

1


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

### Twogram In Threegram

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331836,polygamisch,5
331837,shiit,5
331838,ambient,5
331839,zinswendingen,5


In [6]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_file = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,ik,22539531.0,ik heb,1113451.0,wat is er,364876,wat is er met,43390.0,wat is er met je,12627.0,wat is er,106497.0
1,je,20769946.0,het is,1102152.0,het is een,135216,dat is het niet,14604.0,dat is niet wat ik,2548.0,wat is dat,62768.0
2,het,15696161.0,dat is,939846.0,ik heb een,110642,is er met je,12640.0,het is niet wat je,2357.0,wat was dat,20127.0
3,de,15258816.0,wat is,698111.0,ik heb het,107020,het is maar een,10266.0,wat is er met de,1716.0,wat is het,18734.0
4,dat,13387137.0,heb je,691326.0,wat is dat,92873,dat heb ik niet,9969.0,je dat ik dat niet,1503.0,natuurlijk niet,18232.0
...,...,...,...,...,...,...,...,...,...,...,...,...
83140,NaN,NaN,NaN,NaN,auto die voor,3,NaN,NaN,NaN,NaN,NaN,NaN
83141,NaN,NaN,NaN,NaN,wat voor jasje,3,NaN,NaN,NaN,NaN,NaN,NaN
83142,NaN,NaN,NaN,NaN,wat voor ingewikkeld,3,NaN,NaN,NaN,NaN,NaN,NaN
83143,NaN,NaN,NaN,NaN,de croissant in,3,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [8]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [9]:
#df_shared_count = word_count_result(df_shared_file,["threegram"])
#df_shared_count

In [10]:
df_shared_file["twogram"].nunique()

32877

In [11]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

,twogram,threegram
0,ik heb,ik heb een
1,ik heb,ik heb het
2,ik heb,ik heb je
3,ik heb,ik heb de
4,ik heb,ik heb er
...,...,...
139050,hoogtevrees in,hoogtevrees in het
139051,hoogtevrees in,ik hoogtevrees in
139052,antibiotica verband,antibiotica verband en
139053,antibiotica van,de antibiotica van


In [12]:
df_two_in_three["twogram"].nunique()

25700

In [13]:
df_shared_select_twogram = df_shared_file.loc[:,["twogram","freq_twogram"]]
df_shared_select_twogram

,twogram,freq_twogram
0,ik heb,1113451.0
1,het is,1102152.0
2,dat is,939846.0
3,wat is,698111.0
4,heb je,691326.0
...,...,...
83140,NaN,NaN
83141,NaN,NaN
83142,NaN,NaN
83143,NaN,NaN


In [14]:
set_shared_twogram = set(df_shared_select_twogram["twogram"])
set_two_three = set(df_two_in_three["twogram"])

In [15]:
df_twogram_in_threegram = pd.DataFrame(set_two_three, columns=["twogram"])  # columns=["twogram_in_threegram"]
df_twogram_in_threegram

,twogram
0,video op
1,gas op
2,het sport
3,type zijn
4,voor actie
...,...
25695,een programma
25696,n evenwicht
25697,n krater
25698,kostuum is


In [16]:
df_twogram_in_threegram_freq = pd.merge(df_twogram_in_threegram, df_shared_select_twogram, how="left", on="twogram")
df_twogram_in_threegram_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_in_threegram_freq.rename(columns={"twogram":"twogram_in_threegram","freq_twogram":"freq_two_in_three"}, inplace=True)
df_twogram_in_threegram_freq.reset_index(drop=True, inplace=True)
df_twogram_in_threegram_freq

,twogram_in_threegram,freq_two_in_three
0,ik heb,1113451.0
1,het is,1102152.0
2,dat is,939846.0
3,wat is,698111.0
4,heb je,691326.0
...,...,...
25695,maestro maar,3.0
25696,fase probleem,3.0
25697,lokaal adres,3.0
25698,was pardon,3.0


In [17]:
df_twogram_diff = pd.DataFrame(set_shared_twogram.difference(set_two_three), columns=["twogram"])
df_twogram_diff

,twogram
0,NaN
1,fitness je
2,voor terreur
3,muzikaal niet
4,en tunnel
...,...
7173,compleet een
7174,is antibiotica
7175,we partner
7176,ik protestant


In [18]:
df_twogram_diff_freq = pd.merge(df_twogram_diff, df_shared_select_twogram, how="left", on="twogram")
df_twogram_diff_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_diff_freq.reset_index(drop=True, inplace=True)
df_twogram_diff_freq

,twogram,freq_twogram
0,voor paniek,233.0
1,op plan,207.0
2,hallo baby,174.0
3,van sector,171.0
4,hallo chef,171.0
...,...,...
57440,NaN,NaN
57441,NaN,NaN
57442,NaN,NaN
57443,NaN,NaN


In [19]:
df_shared_file["twogram"] = df_twogram_diff_freq["twogram"]
df_shared_file["freq_twogram"] = df_twogram_diff_freq["freq_twogram"]

In [20]:
df_shared_twogram_process = pd.concat([df_shared_file,df_twogram_in_threegram_freq], axis=1)
df_shared_twogram_process

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence,twogram_in_threegram,freq_two_in_three
0,ik,22539531.0,voor paniek,233.0,wat is er,364876,wat is er met,43390.0,wat is er met je,12627.0,wat is er,106497.0,ik heb,1113451.0
1,je,20769946.0,op plan,207.0,het is een,135216,dat is het niet,14604.0,dat is niet wat ik,2548.0,wat is dat,62768.0,het is,1102152.0
2,het,15696161.0,hallo baby,174.0,ik heb een,110642,is er met je,12640.0,het is niet wat je,2357.0,wat was dat,20127.0,dat is,939846.0
3,de,15258816.0,van sector,171.0,ik heb het,107020,het is maar een,10266.0,wat is er met de,1716.0,wat is het,18734.0,wat is,698111.0
4,dat,13387137.0,hallo chef,171.0,wat is dat,92873,dat heb ik niet,9969.0,je dat ik dat niet,1503.0,natuurlijk niet,18232.0,heb je,691326.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83140,NaN,NaN,NaN,NaN,auto die voor,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83141,NaN,NaN,NaN,NaN,wat voor jasje,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83142,NaN,NaN,NaN,NaN,wat voor ingewikkeld,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83143,NaN,NaN,NaN,NaN,de croissant in,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_shared_twogram_process.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}2.xlsx", index=False)

##### Concat Result With Comma

In [22]:
df_word_order_twogram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_twogram_process, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_twogram_process, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_twogram_process, "word", "sentence")
df_word_order_twogram_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram_in_threegram") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()
df_word_order_join_twogram_threegram = df_word_order_twogram_threegram.groupby(["word"])["twogram_in_threegram"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence,df_word_order_join_twogram_threegram]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,aambei,aambei n,"een aambei van, heb een aambei, voor een aambe...","ik heb een aambei, het is een aambei, heb een ...",ik heb een aambei van,"als een aambei, het is een aambei","een aambei, aambei van"
1,aankondiging,"school aankondiging, muziek aankondiging","heb een aankondiging, voor de aankondiging, de...","ik heb een aankondiging, een aankondiging van ...","is een aankondiging van het, ik heb een aankon...","ik heb een aankondiging, een aankondiging, wat...","een aankondiging, de aankondiging, aankondigin..."
2,aanpassen,"maar aanpassen, van aanpassen, en aanpassen, s...","n schema aanpassen, het niet aanpassen, met he...","het toilet aanpassen en, toilet aanpassen en d...","het toilet aanpassen en de, met het aanpassen ...","strategie aanpassen, me aanpassen, het is aanp...","je aanpassen, het aanpassen, niet aanpassen, a..."
3,aanpassing,"aanpassing is, wat aanpassing, aanpassing en","is een aanpassing, aanpassing van de, en de aa...","het is een aanpassing, is een aanpassing de, i...",het is een aanpassing de,"het is een aanpassing, een aanpassing, aanpass...","een aanpassing, aanpassing van, de aanpassing,..."
4,aantrekkingskracht,"n aantrekkingskracht, voor aantrekkingskracht,...","de aantrekkingskracht van, is de aantrekkingsk...","dat is de aantrekkingskracht, is de aantrekkin...","is de aantrekkingskracht van een, dat is de aa...","wat is de aantrekkingskracht, het was een verh...","de aantrekkingskracht, aantrekkingskracht van,..."
...,...,...,...,...,...,...,...
1445,uppercut,NaN,"op die uppercut, voor een uppercut, wat een up...",voor een uppercut en,NaN,"een uppercut, wat een uppercut","een uppercut, die uppercut, uppercut en"
1446,veganistisch,NaN,"ze zijn veganistisch, dat veganistisch restaur...","dat ze veganistisch zijn, een atleet een vegan...",was een atleet een veganistisch,"ze zijn veganistisch, we zijn veganistisch","zijn veganistisch, dat veganistisch, ze vegani..."
1447,verhouding,NaN,"een verhouding met, je een verhouding, n verho...","heb je een verhouding, je een verhouding met, ...","heb je een verhouding met, dat ik een verhoudi...","heb je een verhouding, een verhouding, ik heb ...","een verhouding, verhouding met, n verhouding, ..."
1448,veterinair,NaN,"en een veterinair, een veterinair patholoog",en een veterinair patholoog,NaN,veterinair patholoog,"een veterinair, veterinair patholoog"


In [26]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence","twogram_in_threegram"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,ik,22539531,"succes ik, romantisch ik, baron ik, ik majeste...","ik heb een, ik heb het, ik heb je, dat heb ik,...","dat heb ik niet, ik heb het niet, dat is wat i...","dat is niet wat ik, je dat ik dat niet, ik heb...","ik niet, ik heb het, ik heb je, en ik, dat was...","ik heb, dat ik, ik niet, heb ik, wat ik, ik je..."
1,je,20769946,"wauw je, hal je, norm je, maria je, risico je,...","ik heb je, wat heb je, heb je het, je dat ik, ...","is er met je, is het met je, wat heb je met, w...","wat is er met je, het is niet wat je, je dat i...","wat heb je, ik heb je, wat is je naam, wat is ...","heb je, je dat, dat je, je niet, je het, je me..."
2,het,15696161,"het professor, het crimineel, het majesteit, h...","het is een, ik heb het, dat is het, heb je het...","dat is het niet, het is maar een, ik heb het n...","het is niet wat je, je dat ik het niet, maar d...","wat is het, dat is het, ik heb het, het is, da...","het is, het niet, is het, je het, het was, dat..."
3,de,15258816,"meter de, favoriet de, kilometer de, de bioloo...","ik heb de, dat is de, het is de, niet in de, h...","je me in de, je me voor de, me niet in de, hij...","wat is er met de, wat is de naam van, ik heb j...","de politie, de wat, in de auto, wat de, de bab...","in de, van de, is de, op de, voor de, met de, ..."
4,dat,13387137,"dat federatie, hal dat, dat kanselier, dat geo...","wat is dat, dat is niet, dat is het, dat is ee...","dat is het niet, dat heb ik niet, wat is dat v...","dat is niet wat ik, je dat ik dat niet, wat is...","wat is dat, wat was dat, dat is het, dat is he...","dat is, je dat, dat je, is dat, dat ik, dat wa..."
...,...,...,...,...,...,...,...,...
1445,bibliografie,13,en bibliografie,in de bibliografie,NaN,NaN,NaN,de bibliografie
1446,testis,13,"testis kanker, een testis",NaN,NaN,NaN,NaN,NaN
1447,caroteen,12,van caroteen,NaN,NaN,NaN,NaN,NaN
1448,houdstermaatschappij,11,NaN,een internationale houdstermaatschappij,is een internationale houdstermaatschappij,NaN,NaN,internationale houdstermaatschappij


In [27]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}2.xlsx", index=False)

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}2.xlsx")
output_file

['Dutch_Turkish_Shared_Result_With_Frequency12.xlsx',
 'Dutch_Turkish_Shared_Join_Result_Without_Frequency12.xlsx']

In [29]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [30]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Prefix Suffix Result Analysis

In [105]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [106]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Italian"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # always must be True in this part
native_word = False # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

4


In [107]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [108]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [109]:
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [110]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

,search_word,word
0,abone,abone
1,abone,abonelik
2,abone,aboneliği
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4835,şövalye,şövalyenin
4836,şövalye,şövalyesi
4837,şırınga,şırınga
4838,şırınga,şırıngayla


In [111]:
if file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word

,search_word,word
0,abone,abone
1,abone,abonelik
2,abone,aboneliği
3,absorbe,absorbe
4,absürt,absürt
...,...,...
4835,şövalye,şövalyenin
4836,şövalye,şövalyesi
4837,şırınga,şırınga
4838,şırınga,şırıngayla


In [112]:
df_all_word.search_word.nunique()

1558

In [113]:
df_all_word.word.nunique()

4711

In [114]:
df_shared_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_all = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,biraz,1269641.0,milyon dolar,6377,milyon dolardan fazla,279.0,servis modülü reaksiyon kontrol,20.0,radyo televizyon telefon bar video,10.0,sosisli sandviç,245.0
1,fazla,692043.0,biraz fazla,5111,standart kablolu orkestrası,191.0,fazla kaliteli normal banka,20.0,servis modülü reaksiyon kontrol sistemi,10.0,telefon numarası,232.0
2,kontrol,272328.0,telefon numarası,1473,kongre onur madalyası,113.0,projenin rutin kopya mikrofilm,17.0,modülü reaksiyon kontrol sistemi valfleri,10.0,standart prosedür,192.0
3,doktor,266714.0,biraz kahve,1354,milyon amerikan doları,84.0,frenler fren pedalları kablolar,14.0,NaN,NaN,genel alarm,139.0
4,polis,247969.0,milyon dolarlık,1102,milyar dolardan fazla,50.0,alfa alfa bravo eko,13.0,NaN,NaN,biraz fazla,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6765,NaN,NaN,sörf plajı,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6766,NaN,NaN,bebek gazla,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6767,NaN,NaN,sandviçleri paket,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6768,NaN,NaN,bebek fotoğrafı,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [116]:
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")
df_word_order_fourgram = pd.merge(df_word_order_fourgram,df_all_word, how="inner", on="word")
df_word_order_fivegram = pd.merge(df_word_order_fivegram,df_all_word, how="inner", on="word")
df_word_order_sentence = pd.merge(df_word_order_sentence,df_all_word, how="inner", on="word")

In [117]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["search_word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["search_word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["search_word"])["sentence"].apply(", ".join).reset_index()

In [118]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [119]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all

,word,twogram,threegram,fourgram,fivegram,sentence
0,abone,"gazeteye abone, polis abone, gazete aboneliği",NaN,NaN,NaN,"abone listesi, abone numarası"
1,acente,acente kopyası,NaN,NaN,NaN,NaN
2,adaptasyon,atmosfere adaptasyon,NaN,NaN,NaN,NaN
3,adrenalin,"doz adrenalin, adrenalin deposu, adrenalin fiz...",NaN,NaN,NaN,NaN
4,adres,"fatura adresi, posta adresi, adresi kontrol, a...","adresi telefon numarası, adres telefon numaras...",NaN,NaN,kod adresi
...,...,...,...,...,...,...
1108,offshore,NaN,NaN,NaN,NaN,offshore operasyonlar
1109,pelikan,NaN,NaN,NaN,NaN,pelikan flamingo
1110,sübjektif,NaN,NaN,NaN,NaN,biraz sübjektif
1111,termometre,NaN,NaN,NaN,NaN,diferansiyel termometre


In [120]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="left", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, trafik kontrol, kontrol pane...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, rutin kontrol, manuel kontrol, ..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktor avukat, dok...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, fazla polis, polis raporu, a...","metro polis departmanı, polis robot polis, met...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, süper polis, poli..."
3,dolar,199343.0,"milyon dolar, milyar dolar, milyonlarca dolar,...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, milyar dolar,..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"biraz aksi, komisyon aksi, aksine ambulans, ak...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [121]:
df_word_order_join_all.word.nunique()

1113

In [122]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx", index=False)

#### Copy Move And Delete

In [123]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}3.xlsx")
output_file2

['Turkish_Italian_Shared_Join_Result_Without_Frequency43.xlsx']

In [124]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [125]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Prefix Suffix Shared File Word Result Select

In [126]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [127]:
## language pair (same previous part parameter)
#lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word sample
word_sample_num = 20

print(f"{file_ext}")

4


In [128]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [129]:
df_shared_process_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx")
df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, trafik kontrol, kontrol pane...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, rutin kontrol, manuel kontrol, ..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktor avukat, dok...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, fazla polis, polis raporu, a...","metro polis departmanı, polis robot polis, met...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, süper polis, poli..."
3,dolar,199343.0,"milyon dolar, milyar dolar, milyonlarca dolar,...","dolar biraz fazla, milyon dolar fazla, milyon ...",NaN,NaN,"milyonlarca dolar, milyon dolar, milyar dolar,..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"biraz aksi, komisyon aksi, aksine ambulans, ak...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, barmenlik sertifikası, barmen...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [130]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [131]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [132]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [133]:
df_fourgram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")
df_fourgram_all = df_fourgram_all.loc[:,["fourgram","frequency"]]
df_fourgram_all["fourgram"] = df_fourgram_all["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram_all

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [134]:
df_fivegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")
df_fivegram_all = df_fivegram_all.loc[:,["fivegram","frequency"]]
df_fivegram_all["fivegram"] = df_fivegram_all["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram_all

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [135]:
df_sentence_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")
df_sentence_all = df_sentence_all.loc[:,["sentence","frequency"]]
df_sentence_all["sentence"] = df_sentence_all["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence_all

,sentence,frequency
0,evet,1948596
1,fikret,1533918
2,hayır,1250401
3,tamam,882921
4,ne,753105
...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6
2913966,eminim uydurmuştur,6
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6
2913968,tabii gerçek bir profesyonel o,6


In [136]:
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass
    try:
        # column result
        df_four_var = pd.DataFrame(df_shared_process_all.loc[i,"fourgram"].split(", "), columns=["fourgram"])
        # merge with all
        df_four_var_merge = pd.merge(df_four_var, df_fourgram_all, how="left", on="fourgram")
        df_four_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_four_var_merge_select = df_four_var_merge.head(word_sample_num)
        # to list value
        df_four_var_list = df_four_var_merge_select["fourgram"].to_list()
        # list join
        df_four_var_list_join = ", ".join(df_four_var_list)
        # change value
        df_shared_process_all.loc[i,"fourgram"] = df_four_var_list_join
    except:
        pass
    try:
        # column result
        df_five_var = pd.DataFrame(df_shared_process_all.loc[i,"fivegram"].split(", "), columns=["fivegram"])
        # merge with all
        df_five_var_merge = pd.merge(df_five_var, df_fivegram_all, how="left", on="fivegram")
        df_five_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_five_var_merge_select = df_five_var_merge.head(word_sample_num)
        # to list value
        df_five_var_list = df_five_var_merge_select["fivegram"].to_list()
        # list join
        df_five_var_list_join = ", ".join(df_five_var_list)
        # change value
        df_shared_process_all.loc[i,"fivegram"] = df_five_var_list_join
    except:
        pass
    try:
        # column result
        df_sentence_var = pd.DataFrame(df_shared_process_all.loc[i,"sentence"].split(", "), columns=["sentence"])
        # merge with all
        df_sentence_var_merge = pd.merge(df_sentence_var, df_sentence_all, how="left", on="sentence")
        df_sentence_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_sentence_var_merge_select = df_sentence_var_merge.head(word_sample_num)
        # to list value
        df_sentence_var_list = df_sentence_var_merge_select["sentence"].to_list()
        # list join
        df_sentence_var_list_join = ", ".join(df_sentence_var_list)
        # change value
        df_shared_process_all.loc[i,"sentence"] = df_sentence_var_list_join
    except:
        pass      

df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,kontrol,272328.0,"numarayı kontrol, biraz kontrolden, trafik kon...","reaksiyon kontrol sistemi, modülü reaksiyon ko...","servis modülü reaksiyon kontrol, modülü reaksi...","servis modülü reaksiyon kontrol sistemi, modül...","bilet kontrol, sistem kontrolü, rutin kontrol,..."
1,doktor,266714.0,"doktor fane, doktor raporu, doktorun numarası,...",ambulans avukat doktor,"olimpiyat şampiyonu doktor üniversite, şampiyo...",NaN,"doktor fane, doktor park, bravo doktor, alo do..."
2,polis,247969.0,"polis departmanı, trafik polisi, fazla polis, ...","düzine fransız polisi, metro polis departmanı,...",polis istasyonunda alarm otomatik,NaN,"alo polis, polis departmanı, metro polisi, tra..."
3,dolar,199343.0,"milyon dolar, milyon dolarlık, milyar dolar, d...","milyon dolardan fazla, milyon amerikan doları,...",NaN,NaN,"milyonlarca dolar, amerikan doları, milyon dol..."
4,alo,152645.0,"alo polis, alo radyo, alo ajan, alo gene, alo ...",NaN,NaN,NaN,"alo polis, alo doktor, alo operatör, alo radyo..."
...,...,...,...,...,...,...,...
1108,deodoran,11.0,marka deodorant,NaN,NaN,NaN,NaN
1109,aks,NaN,"amerikan aksanı, biraz aksi, aksine ambulans, ...",NaN,standart mekanik aksam standart,NaN,"biraz aksi, amerikan aksanı, fransız aksanı"
1110,barmen,NaN,"barmenlik kursu, otelin barmeni, barmenlik ser...",NaN,NaN,NaN,NaN
1111,ekolayzer,NaN,ekolayzerin modern,NaN,NaN,NaN,NaN


In [137]:
#df_shared_process_all.sort_values(by="frequency", ascending=False, inplace=True)

In [138]:
df_shared_process_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Select_Result_Without_Frequency{file_ext}4.xlsx", index=False)

#### Copy Move And Delete

In [139]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*_Select_*{file_ext}4.xlsx")
output_file3

['Turkish_Italian_Shared_Join_Select_Result_Without_Frequency44.xlsx']

In [140]:
for l in output_file3:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [141]:
for j in output_file3:
    try:
        os.remove(j)
    except:
        pass

### Word Count Analysis

In [ ]:
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [ ]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/Deploy Result Manuel").mkdir(parents=True, exist_ok=True)

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

#### Hybrid Words

In [ ]:
df_hybrid = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram hybrid")
df_hybrid

In [ ]:
df_hybrid_count = word_count_result(df_hybrid, ["twogram_pair1","twogram_pair2","twogram_pair3","twogram_pair4"])
df_hybrid_count

In [ ]:
df_hybrid_count_merge = pd.merge(df_hybrid,df_hybrid_count,how="left",on="word")
df_hybrid_count_merge

In [ ]:
df_hybrid_count_merge2 = pd.merge(df_hybrid,df_hybrid_count,how="outer",on="word")
df_hybrid_count_merge2

In [ ]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Hybrid_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_hybrid_count_merge.to_excel(writer, sheet_name='28_Hybrid_Word_Count', index=False)
df_hybrid_count_merge2.to_excel(writer, sheet_name='All_Hybrid_Word_Count', index=False)

In [ ]:
writer.save()

#### Target Words

In [ ]:
df_target = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram target")
df_target

In [ ]:
df_target_count = word_count_result(df_target, ["twogram_1","twogram_2","twogram_3","twogram_4"])
df_target_count

In [ ]:
df_target_count_merge = pd.merge(df_target,df_target_count,how="left",on="word")
df_target_count_merge

In [ ]:
df_target_count_merge2 = pd.merge(df_target,df_target_count,how="outer",on="word")
df_target_count_merge2

In [ ]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Target_Word_Count.xlsx", engine='xlsxwriter')

In [ ]:
df_target_count_merge.to_excel(writer2, sheet_name='28_Target_Word_Count', index=False)
df_target_count_merge2.to_excel(writer2, sheet_name='All_Target_Word_Count', index=False)

In [ ]:
writer2.save()

#### All Target Hybrid Word Count 

In [ ]:
df_target_count

In [ ]:
df_hybrid_count

In [ ]:
df_all_word = pd.concat([df_target_count, df_hybrid_count], axis=0)
df_all_word

In [ ]:
df_all_word.groupby("word")[["word_count"]].sum().reset_index(inplace=True)
df_all_word.sort_values(by="word_count", ascending=False, inplace=True)
df_all_word

In [ ]:
df_all_word.to_excel(f"{lang_folder}_{lang_pair}_Target_Hybrid_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file4 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Word_Count.xlsx")
output_file4

In [ ]:
for o in output_file4:
    source = o # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [ ]:
for p in output_file4:
    try:
        os.remove(p)
    except:
        pass